In [2]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
import torch

from data import get_data

In [4]:
# df = pd.DataFrame([[1,True,'Yes'],[2,True,'Yes'],[3,False,'No'],[4,True,'No'],[5,False,'Yes'],
#                    [6,False,'No'],[7,True,'Yes'],[8,False,'No'],[9,True,'No'],[10,True,'Yes'],
#                    ],
#                    columns=['num','income_over_1000','tv_at_home'])
# df.head()
# goal_label = 'tv_at_home'

In [5]:
# df = get_data('drug200.csv')
# goal_label = 'Drug'

In [6]:
# df = get_data('titanic/train.csv')
# df = df.drop(labels=['Name','Ticket'], axis=1)
# goal_label = 'Survived'


In [42]:
df = pd.DataFrame(data=[[0.5,0.25,'blue'],
                        [1.75,0.6,'blue'],
                        [0.1,1.5,'blue'],
                        [0.8,1.3,'blue'],
                        [1.1,2.4,'blue'],
                        [3,0.5,'green'],
                        [2.5,1.3,'green'],
                        [2.2,1.8,'green'],
                        [2.8,2.1,'green'],
                        [2.5,3,'green'],
                        ],
                  columns=['x','y','colour'])
goal_label = 'colour'

In [43]:
len(df)

10

In [44]:
df.dropna()
len(df)

10

In [45]:
unique_vals = {}
for col in df.columns:
    if col == 'id':
        continue
    col_df = df[col]
    unique_vals[col] = col_df.unique()

In [46]:
unique_vals.keys()

dict_keys(['x', 'y', 'colour'])

In [47]:
input_data = df[[key for key in unique_vals.keys() if key != goal_label]].copy()
labels = df[[goal_label]].copy

In [48]:
input_data

,x,y
0,0.50,0.25
1,1.75,0.60
2,0.10,1.50
3,0.80,1.30
4,1.10,2.40
5,3.00,0.50
6,2.50,1.30
7,2.20,1.80
8,2.80,2.10
9,2.50,3.00


In [49]:
labels()

,colour
0,blue
1,blue
2,blue
3,blue
4,blue
5,green
6,green
7,green
8,green
9,green


In [50]:
class_to_idx = {}
idx_to_class = {}
for key in unique_vals:
    if isinstance(unique_vals[key][0], str):
        num_classes = len(unique_vals[key])
        class_to_idx[key] = {unique_vals[key][idx]:idx for idx in range(num_classes)}
        idx_to_class[key] = {idx: unique_vals[key][idx] for idx in range(num_classes)}
        # one_hot = torch.nn.functional.one_hot(torch.arange(0,num_classes)).float()
        # one_hot_encodings[key] = one_hot
class_to_idx

{'colour': {'blue': 0, 'green': 1}}

In [51]:
idx_to_class

{'colour': {0: 'blue', 1: 'green'}}

In [52]:
for key in class_to_idx:
    df[key] = df[key].map(class_to_idx[key])
df

,x,y,colour
0,0.50,0.25,0
1,1.75,0.60,0
2,0.10,1.50,0
3,0.80,1.30,0
4,1.10,2.40,0
5,3.00,0.50,1
6,2.50,1.30,1
7,2.20,1.80,1
8,2.80,2.10,1
9,2.50,3.00,1


In [63]:
### Information Gain - Classification

def entropy(probs):
    entropy = 0.
    for prob in probs:
        entropy -= (prob)*np.log2(prob) if prob > 0 else 0.
    return entropy


def compute_probs(values: list,
                  tar_vals: list,
                  df: pd.DataFrame,
                  col_idx: str,
                  tar_idx: str,
                  ):
    counts = []
    for val in values:
        tar_counts = []
        for tar_val in tar_vals:
            tar_counts.append(len(df[(df[col_idx]==val) & (df[tar_idx]==tar_val)]))
        counts.append(tar_counts)
    probs = []
    for val_count in counts:
        probs.append([v / sum(val_count) for v in val_count])
    return probs, counts


def bucket_probs(tar_values: list,
                 df: pd.DataFrame,
                 col_idx: str,
                 num_buckets: int = 2,
                 ):
    # num_buckets == 2, average split
    data = df.dropna(subset=[col_idx])
    counts = []
    if num_buckets == 2:
        split_vals = [data[col_idx].min(),data[col_idx].mean()] # ,data[col_idx].max()
    else:
        split_vals = [] # split into n buckets
    for val0,val1 in zip(split_vals[:-1],split_vals[1:]):
        tar_counts = []
        for val in tar_values:
            tar_counts.append(len(data[data[col_idx].between(val0,val1)==val]))
        counts.append(tar_counts)
    probs = []
    for val_count in counts:
        probs.append([v / sum(val_count) for v in val_count])
    return probs, counts, split_vals


def information_gain(df: pd.DataFrame,
                     split_idx: int,
                     tar_idx: int,
                     max_split_nodes: int = 5,
                     ):
    data_num = df[tar_idx].count()
    # parent entropy
    tar_vals = df[tar_idx].unique()
    if len(tar_vals) <= max_split_nodes:
        counts = []
        for val in tar_vals:
            tar_probs = counts.append(len(df[df[tar_idx]==val]))
        tar_probs = counts / data_num
    else: # for regression, we can take the average or bucket  
        pass
    parent_entropy = entropy(tar_probs) # find int to column map in pandas
    # compute split entropies
    split_vals = df[split_idx].unique()
    if len(split_vals) <= max_split_nodes:
        # print(f'raw count for {split_idx}...')
        split_probs, split_counts = compute_probs(values=split_vals,
                                                  tar_vals=tar_vals,  
                                                  df=df, 
                                                  col_idx=split_idx,
                                                  tar_idx=tar_idx,
                                                  )
    else: # for regression, we can take the average or bucket for 'val'
        # print(f'bucket count for {split_idx}...')
        split_probs, split_counts, split_vals = bucket_probs(tar_values=tar_vals,
                                                             df=df,
                                                             col_idx=split_idx,
                                                             num_buckets=2,
                                                             )
    split_entropy = 0.
    for idx, prob in enumerate(split_probs):
        print()
        print(prob)
        print(entropy(prob))
        split_entropy += entropy(prob) * (sum(split_counts[idx]) / data_num)
        print(split_entropy)
    # information gain
    # print(f'#####\nparent entropy: {parent_entropy}\nsplit entropy: {split_entropy}')
    return parent_entropy - split_entropy, split_vals

In [64]:
information_gain(df,split_idx='x',tar_idx='colour')


[0.6, 0.4]
0.9709505944546686
0.9709505944546686


(0.02904940554533142, [0.1, 1.725])

In [88]:
def best_split(df: pd.DataFrame,
               goal_label: str,
               ):
    best_col = ''
    best_ig = 0.
    best_split_vals = []
    for col in df.columns:
        if col not in [goal_label,'PassengerId','Cabin']:
            ig, split_vals = information_gain(df=df, split_idx=col, tar_idx=goal_label)
            if ig > best_ig:
                best_ig = ig
                best_col = col
                best_split_vals = split_vals
            # print(f"{col+' information gain:':35} {ig:.5f}")
    return best_col, np.sort(best_split_vals)

In [89]:
### Gini Index

def gini(probs):
    gini = 0.
    for prob in probs:
        gini -= prob**2
    return 1-gini



In [90]:
class Node:
    def __init__(self, 
                 node_idxs: list,
                 prev_depth: int = 0,
                 split_idx: int = 0,
                 split_vals: tuple = None,
                 is_terminal: bool = False,
                 ):
        self.split_idx = split_idx
        self.split_vals = split_vals
        self.node_idxs = node_idxs
        self.children = []
        self.depth = prev_depth+1
        self.is_terminal = is_terminal


In [91]:
class DecisionTree:
    def __init__(self,
                 data: pd.DataFrame,
                 goal_label: str, 
                 max_depth: int = 5,
                 min_feats: int = 10,
                 num_class: int = 0,
                 ):
        all_idxs = np.array(range(len(data)))
        self.root = Node(node_idxs=all_idxs)
        self.max_depth = max_depth
        
        self.min_feats = min_feats
        self.num_class = num_class

        self.goal_label = goal_label

        self.grow_tree(data=data,
                       )
    
    def grow_tree(self, 
                  data: pd.DataFrame,
                  ):
        # node_queue = [self.root]
        # while len(node_queue) > 0:
        self.split_node(node=self.root,
                        data=data,
                        )
        

    def split_node(self,
                   node: Node,
                   data: pd.DataFrame,
                   ):
        # print(node.split_idx, len(node.node_idxs))
        
        curr_node = node if node else self.root
        if curr_node.depth >= self.max_depth:
            curr_node.is_terminal = True
            # print('Reached maximum depth!')
            return
        
        if len(curr_node.node_idxs) < self.min_feats:
            curr_node.is_terminal = True
            # print('Not enough data features to split node!')
            return
            
        node_data = data.iloc[curr_node.node_idxs]
        
        split_col, split_vals = best_split(df=node_data,
                                           goal_label=self.goal_label,
                                           )
        
        # print(split_col, split_vals)
        for val0,val1 in zip(split_vals[:],np.append(split_vals[1:],split_vals[-1]+1e-6)):
            data_idxs = data[data[split_col].between(val0,val1,inclusive='left')].index.to_numpy()
            child = Node(split_idx=split_col,
                         split_vals=(val0,val1),
                         prev_depth=curr_node.depth,
                         node_idxs=data_idxs,
                         )
            curr_node.children.append(child)
            
            # child_data = data.iloc[data_idxs]
            self.split_node(child,
                            data.drop([split_col], axis=1), #child_data,
                            )
            # print('next child...')
        
    def enumerate(self):
        nodes = [ self.root ]
        while len(nodes) > 0:
            node = nodes.pop(0)
            # print(f"{' '*node.depth}[{node.split_idx}] - [{node.split_vals}]")
            if not node.is_terminal:
                print(f"{' '*node.depth}[{node.split_idx}] - [{node.split_vals}]")
                for child in node.children[::-1]:
                    nodes.insert(0,child)
            else:
                print(f"{' '*node.depth}[{node.split_idx}] (terminal) - [{node.split_vals}]")


In [92]:
dt = DecisionTree(data=df,
                  goal_label=goal_label,
                  max_depth=4,
                  min_feats=50,
                  )

In [93]:
dt.enumerate()

 [0] - [None]
  [Sex] - [(0, 1.0)]
   [Parch] - [(0.0, 0.23570190641247835)]
    [Age] (terminal) - [(5.0, 32.17850287907869)]
    [Age] (terminal) - [(32.17850287907869, 32.17850387907869)]
   [Parch] (terminal) - [(0.23570190641247835, 0.23570290641247835)]
  [Sex] - [(1, 1.000001)]
   [Pclass] - [(1, 2.0)]
    [Age] (terminal) - [(0.92, 38.233440860215055)]
    [Age] (terminal) - [(38.233440860215055, 38.23344186021505)]
   [Pclass] - [(2, 3.0)]
    [Parch] (terminal) - [(0, 1.0)]
    [Parch] (terminal) - [(1, 2.0)]
    [Parch] (terminal) - [(2, 3.0)]
    [Parch] (terminal) - [(3, 3.000001)]
   [Pclass] - [(3, 3.000001)]
    [Age] (terminal) - [(0.42, 25.14061971830986)]
    [Age] (terminal) - [(25.14061971830986, 25.14062071830986)]


In [94]:
np.append(np.array([1,2,3]),4)

array([1, 2, 3, 4])

In [27]:
from sklearn import tree
classifier = tree.DecisionTreeClassifier()

In [28]:
data = df[['Survived','Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]

data = data.dropna()

In [29]:
X = data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
Y = data[['Survived']]

X = X.replace(class_to_idx)

In [32]:
classifier = classifier.fit(X,Y)

In [35]:
x1 = 0
x2 = x1+10

predictions = classifier.predict(X.iloc[x1:x2])
truths = labels().iloc[x1:x2].values

In [36]:
[predictions[idx] - truths[idx][0] for idx in range(len(predictions))]

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]